In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split,RepeatedKFold,GridSearchCV
from sklearn.metrics import accuracy_score, classification_report,roc_auc_score,multilabel_confusion_matrix,roc_curve,accuracy_score, PrecisionRecallDisplay, f1_score
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsClassifier
import time
import pickle

#Import Data
gene_df = pd.read_csv("/content/drive/Shareddrives/5461 Project/data/Challenge_GIN_release_profile_17804library_181query.txt",sep='\t')
challengeA_df = pd.read_csv("/content/drive/Shareddrives/5461 Project/data/ChallengeA_release_GO_BP_20_500_17804library_1106terms.txt",sep='\t')


X = gene_df.iloc[5000:,1:]
y = challengeA_df.iloc[5000:, 1:]
y = y.replace(-1, 0)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Split test data into training and evaluating sets
x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=10
)
no_positives = 0
aucs = []
k_vals = [3,5,10,15]
undersamples = [1000,2000,4000, 8000]
auc_matrix = []
# Optimize for k and undersample size
for k in k_vals:
    print(k)
    under_sample_aucs = []
    for undersample in undersamples:
        print(undersample)
        aucs = []
        for i in range(y_train.shape[1]):
            # Undersample negative class
            y_train_negatives = y_train[y_train.iloc[:,i] == 0].sample(undersample)
            y_train_positives = y_train[y_train.iloc[:,i] == 1]
            y_train_undersample = pd.concat([y_train_negatives, y_train_positives], axis = 0)
            x_train_undersample = x_train.loc[y_train_undersample.index.values]

            # Oversample positive class
            neg, pos = np.bincount(y_train_undersample.iloc[:,i])
#             print(neg, pos)
            sm = SMOTE(random_state=42, k_neighbors= 2)
            X_res, y_res = sm.fit_resample(x_train_undersample, y_train_undersample.iloc[:,i])
            neg, pos = np.bincount(y_res)
#             print(neg, pos)

            knn = KNeighborsClassifier(n_neighbors=k)
            knn.fit(X_res,y_res)

            if sum(y_test.iloc[:,i]) > 0:
                results = knn.predict(x_test)
                fpr, tpr, threshold = roc_curve(y_test.iloc[:,i].values, results)
                auc = roc_auc_score(y_test.iloc[:,i], results)
                accuracy = accuracy_score(y_test.iloc[:,i], results)
                aucs.append(auc)
            else:
                aucs.append(np.nan)
        under_sample_aucs.append(np.nanmean(auc))
    auc_matrix.append(under_sample_aucs)

3
1000


KeyboardInterrupt: ignored

In [ ]:

X = gene_df.iloc[5000:,1:]
y = challengeA_df.iloc[5000:, 1:]
y = y.replace(-1, 0)

# Split test data into training and evaluating sets
x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=10
)
aucs = []
for i in range(y_train.shape[1]):
  # Undersample negative class
  y_train_negatives = y_train[y_train.iloc[:,i] == 0].sample(5000)
  y_train_positives = y_train[y_train.iloc[:,i] == 1]
  y_train_undersample = pd.concat([y_train_negatives, y_train_positives], axis = 0)
  x_train_undersample = x_train.loc[y_train_undersample.index.values]

  # Oversample positive class
  neg, pos = np.bincount(y_train_undersample.iloc[:,i])
  #             print(neg, pos)
  sm = SMOTE(random_state=42, k_neighbors= 2)
  X_res, y_res = sm.fit_resample(x_train_undersample, y_train_undersample.iloc[:,i])
  neg, pos = np.bincount(y_res)
  #             print(neg, pos)

# Parameter Tuning
estimator_KNN = KNeighborsClassifier(algorithm='auto')
parameters_KNN = {
    'n_neighbors': list(range(1,15,5)),
    'leaf_size': list(range(20,40,5)),
    'p': (1,2),
    'weights': ('uniform', 'distance'),
    'metric': ('minkowski', 'chebyshev'),
}
# with GridSearch
grid_search_KNN = GridSearchCV(
    estimator=estimator_KNN,
    param_grid=parameters_KNN,
    scoring = 'roc_auc',
    n_jobs = -1,
    cv = 5
)
grid_search_KNN.fit(x_train,y_train)
y_pred = grid_search_KNN.predict(x_test)
y_pred = y_pred.transpose()
scores = []
for i in range(y.shape[1]):
  if sum(y_test.iloc[:,i]) > 0:
    score = roc_auc_score(y_test.iloc[:,i], y_pred[i])
    print(score)
    scores.append(score)

print(np.mean(scores))
print(grid_search_KNN.best_params_)


KeyboardInterrupt: ignored

In [ ]:
print(grid_search_KNN.best_score_)
print(grid_search_KNN.best_params_)

nan
{'leaf_size': 20, 'metric': 'minkowski', 'n_neighbors': 1, 'p': 1, 'weights': 'uniform'}


In [ ]:

X = gene_df.iloc[5000:,1:]
y = challengeA_df.iloc[5000:, 1:]
y = y.replace(-1, 0)

# Split test data into training and evaluating sets
x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=10
)

tuned_aucs = []
# results appends in transpose
results = []
tuned_predictions = []
for i in range(y_train.shape[1]):
  # # Undersample negative class
  y_train_negatives = y_train[y_train.iloc[:,i] == 0].sample(5000)
  y_train_positives = y_train[y_train.iloc[:,i] == 1]
  y_train_undersample = pd.concat([y_train_negatives, y_train_positives], axis = 0)
  x_train_undersample = x_train.loc[y_train_undersample.index.values]

  # Oversample positive class
  neg, pos = np.bincount(y_train_undersample.iloc[:,i])
  #             print(neg, pos)
  sm = SMOTE(random_state=42, k_neighbors= 2)
  X_res, y_res = sm.fit_resample(x_train_undersample, y_train_undersample.iloc[:,i])
  neg, pos = np.bincount(y_res)
  #             print(neg, pos)

  KNN = KNeighborsClassifier(n_neighbors = 3, algorithm='auto',metric = "minkowski", weights="uniform")
  KNN.fit(X_res, y_res)
  tuned_results = KNN.predict(x_test)
  tuned_predictions.append(tuned_results)
print(results)
for i in range(y.shape[1]):
  if sum(y_test.iloc[:, i]) > 0:
    tuned_aucs.append(roc_auc_score(y_test.iloc[:,i], tuned_predictions[i]))
  else:
    tuned_aucs.append(0)
np.mean(tuned_aucs)
  

[]


0.515295069865861

In [ ]:
# Plot Results
support = []
roc_scores = []
for i, col in enumerate(y.columns):
  if sum(y_test.iloc[:,i]) > 0:
    score = roc_auc_score(y_test.iloc[:, i], tuned_predictions[i])
    neg, pos = np.bincount(y_test.iloc[:, i])
    roc_scores.append(score)
    support.append(pos)
  else:
    roc_scores.append(0)
    support.append(0)

plt.scatter(support, roc_scores)
plt.axhline(y=0.5, color='black')
plt.title('Effect of Data Set Size - KNN')
plt.xlabel('Support')
plt.ylabel('AUROC Score')
plt.show()

support = []
f1_scores = []
for i, col in enumerate(y.columns):
  if sum(y_test.iloc[:,i]) > 0:
    score = f1_score(y_test.iloc[:, i], tuned_predictions[i])
    neg, pos = np.bincount(y_test.iloc[:, i])
    f1_scores.append(score)
    support.append(pos)
  else:
    f1_scores.append(0)
    support.append(0)

f1_go_terms = np.array(f1_scores)
print("Mean F1 Score:", np.mean(f1_go_terms))
ranked_f1_go_terms = np.argsort(f1_go_terms)
for i in ranked_f1_go_terms:
  print(f1_scores[i], y.columns[i])

plt.scatter(support, f1_scores)
plt.axhline(y=0.5, color='black')
plt.title('Effect of Data Set Size - KNN')
plt.xlabel('Support')
plt.ylabel('F1 Score')
plt.show()
print("Mean F1 Score:", np.mean(f1_scores))
roc_score = []
for i, column in enumerate(y.columns):
    try:
        auc = roc_auc_score(y_test.iloc[:,i], tuned_predictions[i])
        roc_score.append(auc)
    except:
        roc_score.append(0)
plt.hist(roc_score, bins=100)
plt.title('KNN ROC AUC Scores')
plt.axvline(x=0.5, color='black')
plt.ylim(0,100)
plt.xlabel('AUC')
plt.ylabel('Count')
plt.show()
print("Mean AUROC Score:", np.mean(np.array(roc_score)))

fig, ax = plt.subplots()
for i, column in enumerate(y_test):
    display = PrecisionRecallDisplay.from_predictions(np.array(y_test[column]), tuned_predictions[i], ax=ax)
    display.ax_.set_title("2-class Precision-Recall curve")
ax.get_legend().remove()
plt.savefig("knn_recall_curve.png", dpi=300)
plt.show()

support = []
f1_scores = []
for i, col in enumerate(y.columns):
  if sum(y_test.iloc[:,i]) > 0:
    score = f1_score(y_test.iloc[:, i], tuned_predictions[i], average = "weighted")
    neg, pos = np.bincount(y_test.iloc[:, i])
    f1_scores.append(score)
    support.append(pos)
  else:
    f1_scores.append(0)
    support.append(0)
print(np.mean(f1_scores))

0.3112299733102367


In [ ]:

X = gene_df.iloc[5000:,1:]
y = challengeA_df.iloc[5000:, 1:]
y = y.replace(-1, 0)
X_eval = gene_df.iloc[:5000, 1:]


eval_df = pd.DataFrame(columns = y.columns)
eval_df.insert(0, "id", gene_df.iloc[:5000,0])
# Make predictions on eval set
for i, col in enumerate(y.columns):
  # Undersample negative class
  y_train_negatives = y[y.iloc[:,i] == 0].sample(5000)
  y_train_positives = y[y.iloc[:,i] == 1]
  y_train_undersample = pd.concat([y_train_negatives, y_train_positives], axis = 0)
  x_train_undersample = X.loc[y_train_undersample.index.values]

  # Oversample positive class
  neg, pos = np.bincount(y_train_undersample.iloc[:,i])
  #             print(neg, pos)
  sm = SMOTE(random_state=42, k_neighbors= 3)
  X_res, y_res = sm.fit_resample(x_train_undersample, y_train_undersample.iloc[:,i])
  neg, pos = np.bincount(y_res)
  #             print(neg, pos)
  KNN = KNeighborsClassifier(algorithm='auto', n_neighbors=10, p=1, metric = "chebyshev", leaf_size=20, weights = "uniform")
  KNN.fit(X_res,y_res)
  y_pred = KNN.predict(X_eval)
  eval_df[col] = y_pred
  print(y_pred)

In [ ]:
kaggle = pd.read_csv("/content/drive/Shareddrives/5461 Project/data/kaggle.csv")
submission = eval_df[kaggle.columns]
submission.to_csv("knn_kaggle.csv", index = False)